In [4]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier 

 Importing Dataset and Exploratry Data Analysis

In [3]:
job_placement = pd.read_excel("Job_Placement_Data_Cleaned.xlsx")
job_placement.head(5)

,gender,ssc_percentage,ssc_board,hsc_percentage,hsc_board,hsc_subject,degree_percentage,undergrad_degree,work_experience,emp_test_percentage,specialisation,mba_percent,status
0,Male,67.00,Others,91.00,Others,Commerce,58.00,Science & Tech,No,55.0,Human Resources,58.80,Placed
1,Male,79.33,Central,78.33,Others,Science,77.48,Science & Tech,Yes,86.5,Finance,66.28,Placed
2,Male,65.00,Central,68.00,Central,Arts,64.00,Comm & Mgmt,No,75.0,Finance,57.80,Placed
3,Male,56.00,Central,52.00,Central,Science,52.00,Science & Tech,No,66.0,Human Resources,59.43,Not Placed
4,Male,85.80,Central,73.60,Central,Commerce,73.30,Comm & Mgmt,No,96.8,Finance,55.50,Placed


In [5]:
job_placement.shape

(215, 13)

In [7]:
#learning some basic statistics like mean, count, percentiles and so on about numeric columns
job_placement.describe()

,ssc_percentage,hsc_percentage,degree_percentage,emp_test_percentage,mba_percent
count,215.000000,215.000000,215.000000,215.000000,215.000000
mean,67.303395,66.333163,66.370186,72.100558,62.278186
std,10.827205,10.897509,7.358743,13.275956,5.833385
min,40.890000,37.000000,50.000000,50.000000,51.210000
25%,60.600000,60.900000,61.000000,60.000000,57.945000
50%,67.000000,65.000000,66.000000,71.000000,62.000000
75%,75.700000,73.000000,72.000000,83.500000,66.255000
max,89.400000,97.700000,91.000000,98.000000,77.890000


In [9]:
#checking for any null values in any column
job_placement.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 215 entries, 0 to 214
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   gender               215 non-null    object 
 1   ssc_percentage       215 non-null    float64
 2   ssc_board            215 non-null    object 
 3   hsc_percentage       215 non-null    float64
 4   hsc_board            215 non-null    object 
 5   hsc_subject          215 non-null    object 
 6   degree_percentage    215 non-null    float64
 7   undergrad_degree     215 non-null    object 
 8   work_experience      215 non-null    object 
 9   emp_test_percentage  215 non-null    float64
 10  specialisation       215 non-null    object 
 11  mba_percent          215 non-null    float64
 12  status               215 non-null    object 
dtypes: float64(5), object(8)
memory usage: 22.0+ KB


<bound method DataFrame.value_counts of      gender  ssc_percentage ssc_board  hsc_percentage hsc_board hsc_subject  \
0      Male           67.00    Others           91.00    Others    Commerce   
1      Male           79.33   Central           78.33    Others     Science   
2      Male           65.00   Central           68.00   Central        Arts   
3      Male           56.00   Central           52.00   Central     Science   
4      Male           85.80   Central           73.60   Central    Commerce   
..      ...             ...       ...             ...       ...         ...   
210    Male           80.60    Others           82.00    Others    Commerce   
211    Male           58.00    Others           60.00    Others     Science   
212    Male           67.00    Others           67.00    Others    Commerce   
213  Female           74.00    Others           66.00    Others    Commerce   
214    Male           62.00   Central           58.00    Others     Science   

     degree